### Pycrash - Impact Momentum Validation

In [ ]:
# %% allow reloading of modules
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
#sys.path.append("D:\\OneDrive\\pycrash")
#sys.path.append("/home/joemcormier/pycrash/")
sys.path.insert(0,'/home/jmc/Documents/pycrash')

In [ ]:
import pycrash
from pycrash.project import Project, project_info, load_project
from pycrash.vehicle import Vehicle
from pycrash.kinematicstwo import KinematicsTwo
from pycrash.model_calcs.carpenter_momentum import IMPC

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy import integrate
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from IPython import get_ipython
from IPython.display import display
get_ipython().run_line_magic('matplotlib', 'qt')

### Load Project Datafile that was created when project was instantiated
- the project object contains information needed to run other parts of Pycrash

#### Get project information - used to import project object

In [ ]:
project_info('validation impact momentum')

#### Load project and print contents

In [ ]:
proj = load_project('validation impact momentum')
proj.show()

### Create Vehicles

##### ToDo: simplify redundant vehicle inputs

In [6]:
# Create Vehicle 1:
vehicle_input_dict = {"year":2016,
"make":"Honda",
"weight":3451,
"init_x_pos":0,
"init_y_pos":0,
"head_angle":0,
"width":6,
"length":19.3,
"hcg":2,
"lcgf":4.88,
"lcgr":6.96,
"wb":11.84,
"track":6.6,
"f_hang":3.2,
"r_hang":4.1,
"tire_d":2.716666667,
"tire_w":0.866666667,
"izz":2152,
"fwd":0,
"rwd":1,
"awd":0,
"L":0,
"c":0,
"vx_initial":10,
"vy_initial":0,
"omega_z":0}

veh1 = Vehicle('Veh1', vehicle_input_dict)

Vehicle inputs for Veh1 applied succesfully


In [ ]:
# %% Create Vehicle 2:
vehicle_input_dict2 = {"year":2016,
"make":"Chevy",
"weight":4695,
"steer_ratio":16.5,
"init_x_pos":25,
"init_y_pos":10,
"head_angle":270,
"width":6,
"length":19.3,
"hcg":2,
"lcgf":4.88,
"lcgr":6.96,
"wb":11.84,
"track":6.6,
"f_hang":3.2,
"r_hang":4.1,
"tire_d":2.716666667,
"tire_w":0.866666667,
"izz":2804,
"fwd":0,
"rwd":1,
"awd":0,
"A":100,
"B":41,
"k":1000,
"L":0,
"c":0,
"vx_initial":15,
"vy_initial":0,
"omega_z":0}

veh2 = Vehicle('Veh2', vehicle_input_dict2)

#### Create Inputs

In [ ]:
test_data = {}
test_data['4363'] = {v1_cg_ic: 2.6,
                     v1_theta_cg_ic: 23.8,
                     v1_head_angle: 180,
                     v1_vx: 35.1,
                     v2_cg_ic: 5.7,
                     v2_theta_cg_ic: 10.6,
                     v2_head_angle: 32,
                     v2_vx: 35}
test_do = '4363'
v1_rotate_theta = (test_data[test_do]['v1_head_angle'] + test_data[test_do]['v1_theta_cg_ic']) * np.pi / 180
veh1.head_angle = test_data[test_do]['v1_head_angle']
veh1.init_x_pos = (test_data[test_do]['v1_cg_ic'] * np.cos(test_data[test_do]['v1_theta_cg_ic'] * np.pi / 180)) * np.cos(v1_rotate_theta) + \
                  (test_data[test_do]['v1_cg_ic'] * np.sin(test_data[test_do]['v1_theta_cg_ic'] * np.pi / 180)) * np.sin(v1_rotate_theta)
veh1.init_y_pos = -1* (test_data[test_do]['v1_cg_ic'] * np.cos(test_data[test_do]['v1_theta_cg_ic'] * np.pi / 180)) * np.sin(v1_rotate_theta) + \
                  (test_data[test_do]['v1_cg_ic'] * np.sin(test_data[test_do]['v1_theta_cg_ic'] * np.pi / 180)) * np.cos(v1_rotate_theta)

# getting initial x,y position of the vehicles.




veh1_input = vehicle_data['FordFusion'].copy()
veh1_input['vx_initial'] = test_inputs['impact_speed']
veh1_input['pimpact_x'] = test_inputs['d_impact'] * np.cos(test_inputs['phi_impact'] * np.pi / 180)
veh1_input['pimpact_y'] = test_inputs['d_impact'] * np.sin(test_inputs['phi_impact'] * np.pi / 180)
veh1_input['impact_norm_rad'] = test_inputs['gamma'] * np.pi / 180

veh2_input = vehicle_data['barrier'].copy()
veh2_input['vx_initial'] = 0
veh2_input['init_x_pos'] = vehicle_data['barrier']['length'] / 2 + vehicle_data['FordFusion']['lcgf'] + \
                           vehicle_data['FordFusion']['f_hang']
veh2_input['init_y_pos'] = test_inputs['d_impact'] * np.sin(test_inputs['phi_impact'] * np.pi / 180)
veh2_input['head_angle'] = 180

# append data to test data dictionary
test_inputs['pimpact_x'] = test_inputs['d_impact'] * np.cos(test_inputs['phi_impact'] * np.pi / 180)
test_inputs['pimpact_y'] = test_inputs['d_impact'] * np.sin(test_inputs['phi_impact'] * np.pi / 180)
test_inputs['impact_norm_rad'] = test_inputs['gamma'] * np.pi / 180
test_inputs['barrier_x_pos'] = vehicle_data['barrier']['length'] / 2 + vehicle_data['FordFusion']['lcgf'] + vehicle_data['FordFusion']['f_hang']
test_inputs['barrier_y_pos'] = test_inputs['d_impact'] * np.sin(test_inputs['phi_impact'] * np.pi / 180)


In [ ]:
sim_inputs = {'cor': res[0][0],
              'cof': res[0][1],
              'impact_norm_deg': res[0][2]}
print(f'Simulation inputs for {current_test}: {sim_inputs.values()}')

run = IMPC(current_test, veh1, veh2, sim_inputs)
run.results()